## Generando formas simples 

Fuente: https://giotto-ai.github.io/gtda-docs/0.5.1/notebooks/classifying_shapes.html

Generemos un conjunto de datos sintéticos de 10 círculos, esferas y toros ruidosos, donde el efecto del ruido es desplazar los puntos que muestrean las superficies en una cantidad aleatoria y en una dirección aleatoria:

In [18]:
from generate_datasets import make_point_clouds

point_clouds_basic, labels_basic = make_point_clouds(n_samples_per_shape=10, n_points=25, noise=0.5)
point_clouds_basic.shape, labels_basic.shape

((30, 625, 3), (30,))

Aquí las etiquetas se definen como que un círculo es 0, una esfera es 1 y un toro es 2. Cada nube de puntos corresponde a una muestra de la superficie continua; en este caso, se muestrean 625 puntos por forma. 
Visualicemos estas nubes de puntos usando la API de trazado de ``giotto-tda``:

In [19]:
from gtda.plotting import plot_point_cloud

plot_point_cloud(point_clouds_basic[0])

In [20]:
plot_point_cloud(point_clouds_basic[10])

In [21]:
plot_point_cloud(point_clouds_basic[-1])

En ``giotto-tda`` podemos derivar diagramas de persistencia a partir de datos seleccionando el transformador deseado en [gtda.homology](https://giotto-ai.github.io/gtda-docs/latest/modules/homology.html) e instanciar la clase como un estimador de ``scikit-learn``. Una vez que se crea una instancia del transformador, podemos hacer uso del paradigma de transformación de ajuste para generar los diagramas:

In [22]:
from gtda.homology import VietorisRipsPersistence

# Seguimiento de componentes conectados, bucles y vacíos
homology_dimensions = [0, 1, 2]

# ¡Contraiga los bordes para acelerar el cálculo de la persistencia H2!
persistence = VietorisRipsPersistence(
    metric="euclidean",
    homology_dimensions=homology_dimensions,
    n_jobs=6,
    collapse_edges=True,
)

diagrams_basic = persistence.fit_transform(point_clouds_basic)

En nuestro ejemplo, cada nube de puntos tiene el mismo número de puntos, por lo que se puede alimentar como una única matriz NumPy. Si tiene un número variable de puntos por nube de puntos, puede pasar una lista de matrices en su lugar.

Una vez que hemos calculado nuestros diagramas de persistencia, podemos comparar cómo el círculo, la esfera y el toro producen diferentes patrones en cada dimensión de homología:

In [23]:
from gtda.plotting import plot_diagram

# Circle
plot_diagram(diagrams_basic[0])

In [24]:
# Sphere
plot_diagram(diagrams_basic[10])

In [25]:
# Torus
plot_diagram(diagrams_basic[-1])

De los diagramas de persistencia podemos ver qué generadores persistentes $H_{1,2,3}$ son más persistentes para cada forma. En particular, vemos que:

* el círculo tiene un generador persistente $H_1$ correspondiente a un bucle,
* la esfera tiene un generador persistente $H_2$ correspondiente a un vacío,
* el toro tiene tres generadores persistentes, dos para $H_1$ y uno para $H_2$.

Aunque los diagramas de persistencia son descriptores útiles de los datos, no se pueden usar directamente para aplicaciones de aprendizaje automático. Esto se debe a que los diferentes diagramas de persistencia pueden tener diferentes números de puntos y las operaciones básicas como la suma y la multiplicación de diagramas no están bien definidas.

Para superar estas limitaciones, se han realizado una variedad de propuestas para "vectorizar" los diagramas de persistencia mediante incrustaciones o núcleos que son adecuados para el aprendizaje automático. En ``giotto-tda``, proporcionamos acceso a las vectorizaciones más comunes a través de la ``gtda.diagrams`` módulo.

Por ejemplo, una de esas características se conoce como [persistence entropy](https://giotto-ai.github.io/gtda-docs/latest/theory/glossary.html#persistence-entropy) que mide la entropía de puntos en un diagrama $D = \{(b_i, d_i)\}_{i\in I}$ de acuerdo a

$$ E(D) = - \sum_{i\in I} p_i \log p_i$$

donde $p_i =(d_i - b_i)/L_D$ and $L_D = \sum_i (d_i - b_i)$. Como hicimos para el cálculo del diagrama de persistencia, podemos usar un transformador para calcular la entropía persistente asociada con cada generador de homología. $H_{0,1,2}$:

In [26]:
from gtda.diagrams import PersistenceEntropy

persistence_entropy = PersistenceEntropy()

# calculate topological feature matrix
X_basic = persistence_entropy.fit_transform(diagrams_basic)

# expect shape - (n_point_clouds, n_homology_dims)
X_basic.shape

(30, 3)

Hemos encontrado una manera de representar cada nube de puntos en términos de solo tres números. Esto no depende del número de puntos en la nube de puntos original, aunque el cálculo de la $H_2$ generadores consume mucho tiempo para las nubes de puntos de $O(10^3)$ puntos. 

Visualizando la matriz de características

In [27]:
plot_point_cloud(X_basic)

vemos que hay tres grupos distintos, lo que sugiere que un clasificador no debería tener problemas para encontrar un límite de decisión limpio.

## Entrena un clasificador

Con nuestras características topológicas a mano, el último paso es entrenar a un clasificador. Dado que nuestro conjunto de datos es pequeño, usaremos un bosque aleatorio y haremos uso de la puntuación OOB para simular la precisión en un conjunto de validación:

In [29]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(oob_score=True)
rf.fit(X_basic, labels_basic)

print(f"OOB score: {rf.oob_score_:.3f}")

OOB score: 1.000


Como era de esperar, nuestro clasificador ha separado perfectamente las 3 clases. A continuación, intentemos combinar todos los pasos como una única canalización.

## Poniendolo todo junto

Evidentemente, hay varios pasos de transformación de datos que deben ejecutarse en el orden correcto para pasar de las nubes de puntos a las predicciones. Por suerte ``giotto-tda`` provee un ``Pipeline`` class para recopilar tales secuencias de transformaciones. A continuación se muestra una pequeña tubería que reproduce todos nuestros pasos:

In [30]:
from gtda.pipeline import Pipeline

steps = [
    ("persistence", VietorisRipsPersistence(metric="euclidean", homology_dimensions=homology_dimensions, n_jobs=6)),
    ("entropy", PersistenceEntropy()),
    ("model", RandomForestClassifier(oob_score=True)),
]

pipeline = Pipeline(steps)

Al llamar al método `` fit ``, la tubería llama a `` fit_transform ``en todos los transformadores antes de llamar a `` fit `` en el estimador final:

In [32]:
pipeline.fit(point_clouds_basic, labels_basic)

Pipeline(steps=[('persistence',
                 VietorisRipsPersistence(homology_dimensions=[0, 1, 2],
                                         n_jobs=6)),
                ('entropy', PersistenceEntropy()),
                ('model', RandomForestClassifier(oob_score=True))])

Luego podemos acceder al modelo Random Forest por él ``model`` clave para elegir la puntuación OOB:

In [33]:
pipeline["model"].oob_score_

0.9666666666666667